In [7]:
import pandas as pd
import numpy as np
import gensim
import tensorflow as tf

In [11]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding

In [2]:
from nltk.corpus import stopwords
import nltk

In [3]:
stopwords = set(stopwords.words('English'))

In [3]:
google_model_path = '../../../LearnSpace/GoogleNews-vectors-negative300.bin.gz'

In [4]:
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 999

In [5]:
train_df = pd.read_csv('../data/sentiment-analysis-on-movie-reviews/train.tsv', sep='\t')

In [6]:
test_df = pd.read_csv('../data/sentiment-analysis-on-movie-reviews/test.tsv', sep='\t')

In [7]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [8]:
test_df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [9]:
train_phrases = train_df['Phrase'].values
train_sentiment = train_df['Sentiment'].values

In [10]:
label_set = np.unique(train_sentiment)
labels = len(label_set)

In [11]:
labels

5

#### Loading the google w2v model

In [84]:
google_w2v = gensim.models.KeyedVectors.load_word2vec_format(google_model_path, binary=True)

#### Input a sentence

In [83]:
test_np = np.vstack((google_w2v['hello'], google_w2v['how'], google_w2v['are'],google_w2v['you']))

In [14]:
# keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', \
#                       embeddings_regularizer=None, activity_regularizer=None, \
#                       embeddings_constraint=None, mask_zero=False, input_length=None)

In [85]:
model = Sequential()
model.add(Embedding(1000, 64, input_length=300))

#### Freezing the embedding layer

In [86]:
model.layers[0].trainable = False

In [87]:
input_array = test_np

In [88]:
model.compile('rmsprop', 'mse')

In [89]:
output_array = model.predict(input_array)

In [90]:
model.output_shape

(None, 300, 64)

In [76]:
output_array

array([[[-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        ...,
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443]],

       [[-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        [-0.03802197,  0.00150628,  0.01148349, ..., -0.01832385,
         -0.00780642,  0.01420443],
        ...,
        [-0.03802197,  0.00150628,  0.01148349, ..., -

In [64]:
test_np = np.vstack((google_w2v['hello'], google_w2v['hi'], google_w2v['fail'],google_w2v['what']))

In [65]:
test_np.ndim

2

In [57]:
test_np.shape

(1200,)

In [58]:
np.array([1,2,3]).shape

(3,)

In [81]:
np.random.randint(1000, size=(5, 10, 2))

array([[[202, 305],
        [833, 629],
        [865, 499],
        [416, 526],
        [767, 967],
        [776, 320],
        [785, 469],
        [508, 550],
        [343, 539],
        [931, 688]],

       [[540, 968],
        [851,  91],
        [ 37, 418],
        [381, 121],
        [718, 510],
        [642, 120],
        [231, 877],
        [997, 526],
        [621, 257],
        [ 66, 100]],

       [[ 56, 466],
        [525, 142],
        [783, 177],
        [565, 166],
        [389, 502],
        [113, 254],
        [308, 549],
        [269, 498],
        [954, 781],
        [192, 448]],

       [[905, 707],
        [994, 650],
        [208, 183],
        [346, 761],
        [ 14, 652],
        [750,  67],
        [428, 523],
        [679, 572],
        [525, 187],
        [ 74, 589]],

       [[553, 761],
        [ 79, 471],
        [561, 674],
        [230, 862],
        [490, 901],
        [ 31, 561],
        [895, 789],
        [906, 402],
        [321, 687],
        [906